In [ ]:
#%pip install tensorflow
%matplotlib widget

In [ ]:
# load data
import numpy as np
import pandas as pd

file_path = "../npz/BTCUSD_1T_close_only.npz"
data = np.load(file_path, allow_pickle=True)
df = pd.DataFrame(data['data'], columns=['timestamp', 'close'])
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['close'] = pd.to_numeric(df['close'], errors='coerce')
df.set_index('timestamp', inplace=True)

In [ ]:
from scipy.ndimage import gaussian_filter1d

n = 50000
dfc = df[:n].copy()
y = dfc['close'].values  # Исходный ряд

sma = pd.Series(y).rolling(256, min_periods=1).mean().values
# Y = (y_smooth - sma) / sma
y = (y - sma) / sma
y_smooth = gaussian_filter1d(y, sigma=8.383354) # 8.383354 corresponds to window size 51 for savgol

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(range(len(y)), (y + 1) * sma, label="y", color="blue", alpha=0.6)
plt.plot(range(len(y)), (y_smooth + 1) * sma, label="y_smooth", color="red", alpha=0.6)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


# Функция генерации признаков (разности скользящих средних)
def generate_features(series, windows=[2, 4, 8, 16, 32, 64, 128, 256]):
    smas = [pd.Series(series).rolling(w, min_periods=1).mean().values for w in windows]
    diffs = [smas[i] - smas[i + 1] for i in range(len(smas) - 1)]
    return np.stack(diffs, axis=-1)  # (количество точек, количество фич)

def generate_ema_features(series, windows=[2, 4, 8, 16, 32, 64, 128, 256]):
    smas = [pd.Series(series).rolling(w, min_periods=1).mean().values for w in windows]
    diffs1 = [smas[i] - smas[i + 1] for i in range(len(smas) - 1)]
    emas = [pd.Series(series).ewm(span=w, adjust=False).mean().values for w in windows]
    diffs2 = [emas[i] - emas[i + 1] for i in range(len(emas) - 1)]
    diffs = np.concatenate([diffs1, diffs2])
    return np.stack(diffs, axis=-1)

# Генерируем X
X = generate_ema_features(y)
Y = y_smooth

# Разделяем на train/test
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]

# Проверяем размерность данных
print(f"X shape: {X.shape}")
print(f"Y shape: {Y.shape}")

# Создаем модель нейросети - исправляем размерность Input
model = keras.Sequential([
    layers.Input(shape=(X.shape[1],)),  # Убираем неверное измерение
    layers.Dense(16, activation="relu"),
    layers.Dense(8, activation="relu"),
    layers.Dense(1)
])

model.compile(optimizer="adam", loss="mse")
model.summary()

# Обучаем модель
history = model.fit(X_train, Y_train, epochs=50, batch_size=16, validation_data=(X_test, Y_test), verbose=1)

# Предсказание на тестовых данных
Y_pred = model.predict(X_test)

# Восстанавливаем предсказанное сглаживание
Y_pred_restored = (Y_pred.flatten() + 1) * sma[-len(Y_pred):]

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(len(Y_test)), (y[-len(Y_test):] + 1) * sma[-len(Y_test):], label="Исходный ряд", color="gray", alpha=0.6)
plt.plot(range(len(Y_test)), (Y_test + 1) * sma[-len(Y_test):], label="Настоящее сглаживание", color="black", alpha=0.6)
plt.plot(range(len(Y_test)), Y_pred_restored, label="Предсказанное нейросетью сглаживание", color="red", alpha=0.8)

# Добавляем скользящие средние разной длины
# sma_windows = [8, 16, 32]
# for w in sma_windows:
#     ma = pd.Series(y).rolling(w, min_periods=1).mean().values[-len(Y_test):]
#     plt.plot(range(len(Y_test)), ma, label=f"SMA {w}", linestyle="dashed", alpha=0.7)

plt.legend()
plt.title("Нейросетевой аналог (без заглядывания в будущее) + скользящие средние")
plt.show()

# Второй график - Y_test и прогноз
plt.figure(figsize=(12, 6))
plt.plot(range(len(Y_test)), Y_test, label="Y_test (целевое сглаживание)", color="black", alpha=0.6)
plt.plot(range(len(Y_test)), Y_pred, label="Предсказанное нейросетью", color="red", alpha=0.8)
plt.legend()
plt.title("Сравнение Y_test и прогноза нейросети")
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(len(Y_test)), y[-len(Y_test):], label="Исходный ряд", color="gray", alpha=0.6)
plt.plot(range(len(Y_test)), Y_test, label="Настоящее сглаживание", color="black", alpha=0.6)
plt.plot(range(len(Y_test)), Y_pred, label="Предсказанное нейросетью сглаживание", color="red", alpha=0.8)